In [1]:
import sys; sys.path.insert(0, '..') # So that we import the local copy of pyzx if you have installed from Github
import os

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

from multiprocessing import Pool
import pyzx as zx
from pyzx import cnot_mapper, architecture # Note that this is the local pyzx package from this repository/fork, not the one installed through pip or other means.
from pyzx import circuit
from pyzx import *
from pyzx.parity_maps import CNOT_tracker, build_random_parity_map
from pyzx.linalg import Mat2
from pyzx.simplify import full_reduce

Load comb circuits

In [2]:
def extract_sub_matrix(matrix, index_list):
    new_matrix = Mat2(np.zeros([len(index_list), matrix.cols()], dtype=int))
    for row in range(new_matrix.rows()):
        for col in range(new_matrix.cols()):
            new_matrix.data[row][col] = matrix.data[index_list[row]][col]
    return new_matrix
def insert_sub_matrix(matrix, sub_matrix, index_list):
    for row in range(sub_matrix.rows()):
        for col in range(sub_matrix.cols()):
            matrix.data[index_list[row]][col] = sub_matrix.data[row][col]
    return matrix
def remove_rowcol(matrix, index):
    matrix.data = np.delete(matrix.data, index, 0)
    matrix.data = np.delete(matrix.data, index, 1)
    print(matrix)

from IPython.display import display, Markdown

DEBUG = False
OUTER_DISPLAY = False
INNTER_DISPLAY = False

#INDEX = 16
equal = 0
for INDEX in range(100):
    circ = loaded_circuits[INDEX].copy()
    OUTER_DISPLAY and display(zx.draw(circ))
    decomposition = CombDecomposition.from_circuit(circ)
    comb = decomposition.comb
    OUTER_DISPLAY and display(zx.draw(comb))
    new_comb = CNOTComb(comb.qubits, comb.holes, comb.new_to_old_qubit_mappings)
    arch = architecture.create_architecture(architecture.LINE, n_qubits=circ.qubits)
    # Create a copy of the parity matrix of the comb to perform gaussian elimination on
    matrix = comb.matrix.copy()


    DEBUG and print(comb.matrix)


    # Find initial qubits for the sub matrix
    qubits_in_matrix = []
    old_to_new_qubits = dict([(i,[]) for i in range(circ.qubits)])
    for virtual_qubit in comb.new_to_old_qubit_mappings.keys():
        old_to_new_qubits[comb.new_to_old_qubit_mappings[virtual_qubit]].append(virtual_qubit)
    for logical_qubit in old_to_new_qubits.keys():
        if len(old_to_new_qubits[logical_qubit]) == 0:
            qubits_in_matrix.append(logical_qubit)
        else:
            qubits_in_matrix.append(max(old_to_new_qubits[logical_qubit]))    

    DEBUG and print(qubits_in_matrix)


    # These are the qubits still accessible on the architecture
    # This doesn't include virtual qubits
    rows_to_eliminate = list(range(circ.qubits))
    cols_to_eliminate = elimination_order(comb)
    while 0 < len(cols_to_eliminate):

        # Generate sub matrix
        sub_matrix = extract_sub_matrix(matrix, qubits_in_matrix)
        sub_circuit = CNOT_tracker(circ.qubits, parities_as_columns=False)
        col_to_eliminate = cols_to_eliminate[0]
        DEBUG and print(comb.holes)
        DEBUG and print(comb.new_to_old_qubit_mappings)

        # Take account of mapping from virtual to logical qubits
        if col_to_eliminate in comb.new_to_old_qubit_mappings:
            row_to_eliminate = comb.new_to_old_qubit_mappings[col_to_eliminate]
        else:
            row_to_eliminate = col_to_eliminate

        DEBUG and print(f"Qubit to eliminate {col_to_eliminate} ({row_to_eliminate})")

        # Remove current rowcol from matrix
        rowcol_iteration(sub_matrix, # need to generate correct sub matrix
                         arch,
                         row_to_eliminate, # rowcol needs to 'see' the virtual qubit as the original
                         col_to_eliminate,
                         rows_to_eliminate, # need to generate list of qubits currently accessible
                         cols_to_eliminate,
                         circuit=sub_circuit)
        DEBUG and print(sub_circuit.gates)

        # Convert the gates of the sub matrix using the mapping
        for gate in sub_circuit.gates[::-1]:
            gate.control = qubits_in_matrix[gate.control]
            gate.target = qubits_in_matrix[gate.target]
            # Add gates from sub circuit to comb circuit
            new_comb.gates.insert(0, gate)
        # Insert submatrix back into larger matrix
        insert_sub_matrix(matrix, sub_matrix, qubits_in_matrix)    

        DEBUG and print(new_comb.gates)


        # If the qubit just removed maps to another qubit via a hole
        # replace that qubit with the new qubit in the qubits_in_matrix list
        qubit_found = False
        qubit_loc = 0
        while not qubit_found:
            qubit = qubits_in_matrix[qubit_loc]
            if qubit == cols_to_eliminate[0]:
                qubit_found = True
                # Remove virtual qubit
                cols_to_eliminate.pop(0)
                # Check if there are anymore virtual qubits that could be assigned to this logical one
                if qubit in comb.holes.inverse.keys():
                    qubits_in_matrix[qubit_loc] = comb.holes.inverse.pop(qubit)
                else:
                    # If no more virtual qubits can be assigned we remove it from being considered
                    # available in the architecture
                    if qubit in comb.new_to_old_qubit_mappings:
                        rows_to_eliminate.remove(comb.new_to_old_qubit_mappings[qubit])
                    else:
                        rows_to_eliminate.remove(qubit)
            qubit_loc += 1

        INNTER_DISPLAY and display(zx.draw(new_comb))      

    decomposition.comb = new_comb
    new_circuit = CombDecomposition.to_circuit(decomposition)
    #print("New Circuit: ")
    #print(f"{INDEX} : {new_circuit.verify_equality(loaded_circuits[INDEX])}")
    equal += new_circuit.verify_equality(loaded_circuits[INDEX])
    OUTER_DISPLAY and display(zx.draw(new_circuit))
print(equal)

In [3]:
def combrowcol(circuit, arch, *args, **kwargs):
    circ = circuit.copy()
    OUTER_DISPLAY and display(zx.draw(circ))
    decomposition = CombDecomposition.from_circuit(circ)
    comb = decomposition.comb
    OUTER_DISPLAY and display(zx.draw(comb))
    new_comb = CNOTComb(comb.qubits, comb.holes, comb.new_to_old_qubit_mappings)
    # Create a copy of the parity matrix of the comb to perform gaussian elimination on
    matrix = comb.matrix.copy()

    DEBUG and print(comb.matrix)

    # Iterate over the gates in the comb and log with qubits are connected via CNOTs
    qubit_connections = dict([(i,[]) for i in range(comb.qubits)])
    for gate in comb.gates:
        # What we want to be doing here is not just adding connected control to target
        # but connecting control to all the qubits connected to target
        for q in qubit_connections[gate.target]:
            if gate.control not in qubit_connections[q]:
                qubit_connections[q].append(gate.control)
        for q in qubit_connections[gate.control]:
            if gate.target not in qubit_connections[q]:
                qubit_connections[q].append(gate.target)
        
        if gate.control not in qubit_connections[gate.target]:
            qubit_connections[gate.target].append(gate.control)
        if gate.target not in qubit_connections[gate.control]:
            qubit_connections[gate.control].append(gate.target)
            
        #qubit_connections[gate.target]  = list(set(qubit_connections[gate.target]).union(set(qubit_connections[gate.control])))
        #qubit_connections[gate.control] = list(set(qubit_connections[gate.target]).union(set(qubit_connections[gate.control])))

    #print(comb.matrix)
    #print(f"Gates : {comb.gates}")
    
    # Find initial qubits for the sub matrix
    qubits_in_matrix = []
    old_to_new_qubits = dict([(i,[]) for i in range(circ.qubits)])
    for virtual_qubit in comb.new_to_old_qubit_mappings.keys():
        old_to_new_qubits[comb.new_to_old_qubit_mappings[virtual_qubit]].append(virtual_qubit)
    for logical_qubit in old_to_new_qubits.keys():
        if len(old_to_new_qubits[logical_qubit]) == 0:
            qubits_in_matrix.append(logical_qubit)
        else:
            qubits_in_matrix.append(max(old_to_new_qubits[logical_qubit]))    

    DEBUG and print(qubits_in_matrix)


    # These are the qubits still accessible on the architecture
    # This doesn't include virtual qubits
    rows_to_eliminate = list(range(circ.qubits))
    cols_to_eliminate = list(range(comb.qubits))
    while 0 < len(cols_to_eliminate):        

        # Generate sub matrix
        sub_matrix = extract_sub_matrix(matrix, qubits_in_matrix)
        sub_circuit = CNOT_tracker(circ.qubits, parities_as_columns=False)
        col_to_eliminate = next_elimination(qubit_connections, qubits_in_matrix, cols_to_eliminate)
        DEBUG and print(comb.holes)
        DEBUG and print(comb.new_to_old_qubit_mappings)

        #print(f"Qubit Connections : {qubit_connections}")
        #print(f"Col to eliminate : {col_to_eliminate}")
        
        # Take account of mapping from virtual to logical qubits
        if col_to_eliminate in comb.new_to_old_qubit_mappings:
            row_to_eliminate = comb.new_to_old_qubit_mappings[col_to_eliminate]
        else:
            row_to_eliminate = col_to_eliminate

        DEBUG and print(f"Qubit to eliminate {col_to_eliminate} ({row_to_eliminate})")

        # Remove current rowcol from matrix
        rowcol_iteration(sub_matrix, # need to generate correct sub matrix
                         arch,
                         row_to_eliminate, # rowcol needs to 'see' the virtual qubit as the original
                         col_to_eliminate,
                         rows_to_eliminate, # need to generate list of qubits currently accessible
                         cols_to_eliminate,
                         circuit=sub_circuit)
        DEBUG and print(sub_circuit.gates)

        # Convert the gates of the sub matrix using the mapping
        for gate in sub_circuit.gates[::-1]:
            gate.control = qubits_in_matrix[gate.control]
            gate.target = qubits_in_matrix[gate.target]
            # Add gates from sub circuit to comb circuit
            new_comb.gates.insert(0, gate)
        # Insert submatrix back into larger matrix
        insert_sub_matrix(matrix, sub_matrix, qubits_in_matrix)    

        DEBUG and print(new_comb.gates)


        # If the qubit just removed maps to another qubit via a hole
        # replace that qubit with the new qubit in the qubits_in_matrix list
        qubit_found = False
        qubit_loc = 0
        while not qubit_found:
            qubit = qubits_in_matrix[qubit_loc]
            if qubit == col_to_eliminate:
                qubit_found = True
                # Remove virtual qubit
                cols_to_eliminate.remove(col_to_eliminate)
                # Check if there are anymore virtual qubits that could be assigned to this logical one
                if qubit in comb.holes.inverse.keys():
                    qubits_in_matrix[qubit_loc] = comb.holes.inverse.pop(qubit)
                else:
                    # If no more virtual qubits can be assigned we remove it from being considered
                    # available in the architecture
                    if qubit in comb.new_to_old_qubit_mappings:
                        rows_to_eliminate.remove(comb.new_to_old_qubit_mappings[qubit])
                    else:
                        rows_to_eliminate.remove(qubit)
            qubit_loc += 1
        # Remove the qubit that has just been eliminated from the connections
        qubit_connections[col_to_eliminate] = []
        for qubit in qubit_connections:
            if col_to_eliminate in qubit_connections[qubit]:
                qubit_connections[qubit].remove(col_to_eliminate)

        INNTER_DISPLAY and display(zx.draw(new_comb))      

    decomposition.comb = new_comb
    new_circuit = CombDecomposition.to_circuit(decomposition)
    OUTER_DISPLAY and display(zx.draw(new_circuit))
    return new_circuit

In [4]:
def elimination_order(comb, qubits):
    #return [3,2,0,1]
    pass

def next_elimination(qubit_connections, qubits_in_matrix, cols_to_eliminate):
    Pnt = False
    qubit_found = False
    qubit_index = 0
    qubit_to_eliminate = None
    
    Pnt and print(f"Qubit Connections : {qubit_connections}")
    Pnt and print(f"Qubit in Matrix : {qubits_in_matrix}")
    
    available_qubits = list(set(qubits_in_matrix) & set(cols_to_eliminate))
    #print(f"Available Qubits : {available_qubits}")
    
    while not qubit_found and qubit_index < len(available_qubits):
        # If if one of he currently accessible qubits is only connected to qubits that are also
        # currently accessible
        
        Pnt and print(f"Considering Qubit : {available_qubits[qubit_index]}")
        Pnt and print(f"Length : {len(qubit_connections[available_qubits[qubit_index]])}")
        Pnt and print(set(qubit_connections[available_qubits[qubit_index]]).issubset(set(available_qubits)))
        
        if set(qubit_connections[available_qubits[qubit_index]]).issubset(set(available_qubits)):
            qubit_found = True
            qubit_to_eliminate = available_qubits[qubit_index]
        qubit_index += 1
    Pnt and print(f"Qubit to Eliminate : {qubit_to_eliminate}")
    return qubit_to_eliminate
    

def next_elimination(sub_matrix, qubits_in_matrix, cols_to_eliminate):
    qubit_found = False
    rows = range(sub_matrix.data.shape[0])
    cols = range(sub_matrix.data.shape[1])
    index = 0
    #print(f"Sub Matrix : \n {sub_matrix}")
    #print(f"Qubits in matrix : \n {qubits_in_matrix}")
    M = sub_matrix.copy()
    M.gauss(full_reduce=True)
    possible_eliminations = []
    
    for index in range(len(qubits_in_matrix)):
        col = qubits_in_matrix[rows[index]]
        
        if col in cols_to_eliminate:
            # Get all the rows that have a 1 in the desired column
            ones = [r for r in rows if M.data[r][col] == 1]
            # Check that there is only one row with a 1 in it
            if len(ones) == 1:
                row = ones.pop()
                # Check that the row only has one one in it
                if sum([M.data[row][c] for c in cols]) == 1:
                    possible_eliminations.append(col)                
    #print(f"M : \n{M}")
    #print(f"Possible Eliminations : {possible_eliminations}")
    return possible_eliminations[0]
        

In [13]:
comb_circuit_folder = "../circuits/combs/"

n_qubits = [4]
n_cnots = [8]
n_non_cnots = [1]
loaded_circuits = {}
for qubits in n_qubits:
    for cnots in n_cnots:
        for non_cnots in n_non_cnots:
            circuit_properties = {"Qubits"    : qubits,
                                  "CNOTs"     : cnots,
                                  "Non-CNOTs" : non_cnots}
            path = os.path.join(comb_circuit_folder,
                                str(circuit_properties["Qubits"])+"qubits", 
                                str(circuit_properties["CNOTs"])+"cnots", 
                                str(circuit_properties["Non-CNOTs"])+"non-cnots")
            loaded_circuits[f"({qubits},{cnots},{n_non_cnots})"] = []
            if os.path.exists(path):
                for file_name in os.listdir(path):
                    loaded_circuits[f"({qubits},{cnots},{n_non_cnots})"].append(circuit.Circuit.from_qasm_file(os.path.join(path, file_name)))
            else:
                print("Directory Does Not Exist")

In [17]:
from IPython.display import display, Markdown

DEBUG = False
OUTER_DISPLAY = False
INNTER_DISPLAY = False

for qubits in n_qubits:
    for cnots in n_cnots:
        for non_cnots in n_non_cnots:
            arch = architecture.create_architecture(architecture.FULLY_CONNNECTED, n_qubits=qubits)
            equal = 0
            for INDEX in [92]: #range(100):
                circ = loaded_circuits[f"({qubits},{cnots},{n_non_cnots})"][INDEX]
                new_circ = combrowcol(circ, arch)
                equality = new_circ.verify_equality(circ)
                equal += equality
                print(f"INDEX: {INDEX} {equality}")
            print(f"Qubits : {qubits}, CNOTs : {cnots}, Non-CNOTs : {non_cnots} -> {equal}")

INDEX: 92 False
Qubits : 4, CNOTs : 8, Non-CNOTs : 1 -> 0
